In [28]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse
from litellm import completion
import litellm
import os
litellm.enable_json_schema_validation = True
litellm.set_verbose = True
import json
from litellm import ModelResponse
from dotenv import load_dotenv
load_dotenv()

df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)

In [29]:
# df = pd.read_csv('50 rows.csv')

In [31]:
df.head(2)

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json


In [32]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

class AnswerCorrectness(BaseModel):
  correct:bool



In [33]:
def process_math_problems(df, start_idx=0, end_idx=20):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        response: ChatResponse = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }],
            response_format=Answer
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [34]:
process_math_problems(df,start_idx=0,end_idx=50)

15:05:05 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {'title': 'Answer', 'type': 'string'}}, 'required': ['reasoning', 'answer'], 'title': 'Answer', 'type': 'object', 'additionalProperties': False}, 'name': 'Answer', 'strict': True}}, 'extra_body': {}}


15:05:07 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03R3iJVbstAxL5B79J9IUAAMquhr", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"To convert the point (0,3) from rectangular coordinates to polar coordinates, we calculate the radius as r = sqrt(x^2 + y^2) = sqrt(0^2 + 3^2) = sqrt(9) = 3. The angle \u03b8 is given by atan2(y, x) = atan2(3, 0) = \u03c0/2 because the point is on the positive y-axis.\",\"answer\":\"(3, \u03c0/2)\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739352905, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 103, "prompt_tokens": 232, "total_tokens": 335, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens"

15:05:08 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{"id": "chatcmpl-B03TcXKqbPcDGg5TThLCVHOjBNdZE", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"Half of $3x-9$ is equal to $(3x-9)/2$. So, setting $(3x-9)/2 = x+37$. Solving for x gives x=61.\",\"answer\":\"x is 61.\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353064, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 56, "prompt_tokens": 209, "total_tokens": 265, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json

15:07:45 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03Td7uaX3tQRgHEUVjd5wdpyW5Yu", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353065, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 117, "total_tokens": 123, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


Processed row 49
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {'

15:07:48 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03TesbpfzGN6Krkr5E45KQsqWLoU", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"Let x be the number of coins in each of the initial 7 bags. The total number of coins originally is 7x. When you find the 53 coin bag, you have a total of 7x + 53 coins. You want to reorganize them into 8 bags, so the total number of coins is divisible by 8. Additionally, 7x + 53 > 200 since you have more than 200 coins. The smallest x for which 7x + 53 is divisible by 8 and exceeds 200 is x = 23. In this case, the number of coins is 7*23 + 53 = 161 + 53 = 214 coins.\",\"answer\":\"The smallest number of coins you could have had is 214.\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353066, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_076d6600c1", "usage": {"completion_tokens": 16

In [35]:
df.to_csv("50 rows gpt4o.csv")

In [36]:
def generate_training_messages(df, start_idx=0, num_rows=20):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [37]:
training_data = generate_training_messages(df)

In [38]:
len(training_data)

81

In [39]:
def test_process_math_problems_te(df, start_idx=20, end_idx=30):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        messages = training_data + [
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }]
        
        response = completion(model='gpt-4o', messages=messages,
            response_format=Answer
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response_test'] = answer_content
        df.at[idx, 'llm_answer_test'] = answer
        df.at[idx, 'is_correct_test'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [40]:
test_process_math_problems_te(df,start_idx=20,end_idx=50)

15:07:49 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {'title': 'Answer', 'type': 'string'}}, 'required': ['reasoning', 'answer'], 'title': 'Answer', 'type': 'object', 'additionalProperties': False}, 'name': 'Answer', 'strict': True}}, 'extra_body': {}}


15:07:51 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03ThWDf7190WdIwFNnwIEk5tmmLx", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"To evaluate the expression, distribute the 6 and combine like terms: \\\\((1 + 2i)6 - 3i = 6 + 12i - 3i = 6 + 9i\\\\).\",\"answer\":\"6 + 9i\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353069, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_076d6600c1", "usage": {"completion_tokens": 61, "prompt_tokens": 8278, "total_tokens": 8339, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type

15:07:52 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{"id": "chatcmpl-B03VOBfZmBtKfpKVdhv1iVJ3poQfw", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"Set up the equation for half of 3x-9 being equal to x+37: (1/2)(3x - 9) = x + 37. Solve for x by first eliminating the fraction, and then isolating x to find x = 61.\",\"answer\":\"x = 61.\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353174, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 70, "prompt_tokens": 8291, "total_tokens": 8361, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 8192}}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final r

15:09:36 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03VQuPV7d32L5mh1iuwYHJc3HJdE", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353176, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 117, "total_tokens": 123, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


Processed row 49
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {'

15:09:39 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B03VR6YAq4yH2fHJmXdPSKMnuwbjV", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"Let x be the number of coins in each of the original seven bags. The total number of coins is more than 200 and divisible by 8 after redistributing to have 8 equal bags. Initially, 7x + 53 is the total number of coins. Solving for conditions stated, we find the smallest possible is 205, such that 7x + 53 = 205 and is least multiple of 8 over 200.\",\"answer\":\"The smallest number of coins is 212.\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739353177, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 109, "prompt_tokens": 8362, "total_tokens": 8471, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_t

In [41]:
df.to_csv('first 50 test gpt4o.csv')

In [42]:
df_test = df.iloc[20:50]

In [43]:
df_test['is_correct'].value_counts()

is_correct
False    17
True     13
Name: count, dtype: int64

In [44]:
df_test['is_correct_test'].value_counts()

is_correct_test
True     15
False    15
Name: count, dtype: int64